In [30]:
print("hello world!!!!")

hello world!!!!


In [31]:
import os, json
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
import numpy as np
from collections import defaultdict

In [32]:
# token
google_token = json.loads(open("tkn.json", "r").read())["google_tkn"]


In [33]:
# creating patient database list
patient_detail_path = "Patient_Details"
patient_list = []
for patient_file in os.listdir(patient_detail_path):
    per_patient_detail = {}
    file = open(os.path.join(patient_detail_path, patient_file), "r")
    for line in file.readlines():
        if line.startswith("Patient_ID"):
            per_patient_detail["patient_id"] = line.split(":")[1].strip()
        if line.startswith("Age"):
            per_patient_detail["age"] = line.split(":")[1].strip()
        if line.startswith("Gender"):
            per_patient_detail["gender"] = line.split(":")[1].strip()
    file = open(os.path.join(patient_detail_path, patient_file), "r")
    for section in file.read().split("--- "):
        if section.startswith("Symptoms"):
            per_patient_detail["symptoms"] = section.split("---")[1].strip().replace("\n", ", ")
        if section.startswith("Vitals"):
            per_patient_detail["vitals"] = section.split("---")[1].strip().replace("\n", ", ")
        if section.startswith("Lab Reports"):
            per_patient_detail["lab_reports"] = section.split("---")[1].strip().replace("\n",", ")
        if section.startswith("Doctor Notes"):
            per_patient_detail["doctor_notes"] = section.split("---")[1].strip().replace("\n", " ")
        if section.startswith("Final Department"):
            per_patient_detail["final_department"] = section.split("---")[1].strip().replace("\n", " ")
        if section.startswith("Outcome"):
            per_patient_detail["outcome"] = section.split("---")[1].strip().replace("\n", " ")
    patient_list.append(per_patient_detail)

print(patient_list)
print(patient_list[0])

[{'patient_id': 'P_10001', 'age': '45', 'gender': 'Male', 'symptoms': 'Chest tightness for 2 days, Mild shortness of breath, Sweating', 'vitals': 'BP: 140/90, HR: 92', 'lab_reports': 'ECG: Normal, Troponin: Normal', 'doctor_notes': 'Cardiac evaluation advised', 'final_department': 'Cardiology', 'outcome': 'OPD evaluation done'}, {'patient_id': 'P_10010', 'age': '55', 'gender': 'Male', 'symptoms': 'Chest pain radiating to left arm, Shortness of breath, Nausea', 'vitals': 'BP: 170/105, HR: 110', 'lab_reports': 'ECG: ST elevation, Troponin: Elevated', 'doctor_notes': 'Acute myocardial infarction suspected', 'final_department': 'Cardiology', 'outcome': 'Emergency angioplasty done'}, {'patient_id': 'P_10011', 'age': '36', 'gender': 'Female', 'symptoms': 'Neck pain, Stiffness, Reduced range of motion', 'vitals': 'BP: 118/76, HR: 74', 'lab_reports': 'X-ray Cervical Spine: Muscle spasm', 'doctor_notes': 'Cervical spondylosis suspected', 'final_department': 'Orthopedics', 'outcome': 'Physiother

In [34]:
patient_list[0]

{'patient_id': 'P_10001',
 'age': '45',
 'gender': 'Male',
 'symptoms': 'Chest tightness for 2 days, Mild shortness of breath, Sweating',
 'vitals': 'BP: 140/90, HR: 92',
 'lab_reports': 'ECG: Normal, Troponin: Normal',
 'doctor_notes': 'Cardiac evaluation advised',
 'final_department': 'Cardiology',
 'outcome': 'OPD evaluation done'}

In [35]:
# creating chunks
chunks = []
for patient in patient_list:
    chunks.append({
        "text": f"Symptoms: {patient['symptoms']}",
        "metadata": {
            "patient_id": patient["patient_id"],
            "chunk_type": "symptoms",
            "department": patient["final_department"]
        }
        })
    chunks.append({
        "text": f"Vitals: {patient['vitals']}\nLab results: {patient['lab_reports']}",
        "metadata": {
            "patient_id": patient["patient_id"],
            "chunk_type": "clinical",
            "department": patient["final_department"]
        }
        })
    chunks.append({
        "text": f"Doctor Assesment: {patient['doctor_notes']}",
        "metadata": {
            "patient_id": patient["patient_id"],
            "chunk_type": "assessment",
            "department": patient["final_department"]
        }
        })

print(chunks)

[{'text': 'Symptoms: Chest tightness for 2 days, Mild shortness of breath, Sweating', 'metadata': {'patient_id': 'P_10001', 'chunk_type': 'symptoms', 'department': 'Cardiology'}}, {'text': 'Vitals: BP: 140/90, HR: 92\nLab results: ECG: Normal, Troponin: Normal', 'metadata': {'patient_id': 'P_10001', 'chunk_type': 'clinical', 'department': 'Cardiology'}}, {'text': 'Doctor Assesment: Cardiac evaluation advised', 'metadata': {'patient_id': 'P_10001', 'chunk_type': 'assessment', 'department': 'Cardiology'}}, {'text': 'Symptoms: Chest pain radiating to left arm, Shortness of breath, Nausea', 'metadata': {'patient_id': 'P_10010', 'chunk_type': 'symptoms', 'department': 'Cardiology'}}, {'text': 'Vitals: BP: 170/105, HR: 110\nLab results: ECG: ST elevation, Troponin: Elevated', 'metadata': {'patient_id': 'P_10010', 'chunk_type': 'clinical', 'department': 'Cardiology'}}, {'text': 'Doctor Assesment: Acute myocardial infarction suspected', 'metadata': {'patient_id': 'P_10010', 'chunk_type': 'asse

In [36]:
chunks[0:4]

[{'text': 'Symptoms: Chest tightness for 2 days, Mild shortness of breath, Sweating',
  'metadata': {'patient_id': 'P_10001',
   'chunk_type': 'symptoms',
   'department': 'Cardiology'}},
 {'text': 'Vitals: BP: 140/90, HR: 92\nLab results: ECG: Normal, Troponin: Normal',
  'metadata': {'patient_id': 'P_10001',
   'chunk_type': 'clinical',
   'department': 'Cardiology'}},
 {'text': 'Doctor Assesment: Cardiac evaluation advised',
  'metadata': {'patient_id': 'P_10001',
   'chunk_type': 'assessment',
   'department': 'Cardiology'}},
 {'text': 'Symptoms: Chest pain radiating to left arm, Shortness of breath, Nausea',
  'metadata': {'patient_id': 'P_10010',
   'chunk_type': 'symptoms',
   'department': 'Cardiology'}}]

In [37]:
# creating embedding
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=google_token)

vector_store = []

for chunk in chunks:
    vector_store.append({
        "embedding": embeddings.embed_query(chunk["text"], task_type="RETRIEVAL_DOCUMENT"),
        "text": chunk["text"],
        "metadata": chunk["metadata"]
        })

In [38]:
vector_store[0]

{'embedding': [-0.014275508932769299,
  -0.023635536432266235,
  0.0035031591542065144,
  -0.09195486456155777,
  0.014115236699581146,
  0.0020582322031259537,
  0.00036807297146879137,
  0.011164884082973003,
  -0.014564908109605312,
  0.020124830305576324,
  0.00755558954551816,
  -0.025715934112668037,
  0.017228147014975548,
  0.010463870130479336,
  0.10900767147541046,
  -0.0042793177999556065,
  -0.02232944592833519,
  0.020234886556863785,
  -0.00017399269563611597,
  -0.021452222019433975,
  0.020864637568593025,
  0.029643211513757706,
  -0.00969141535460949,
  -0.022608689963817596,
  0.008474581874907017,
  -0.01259543001651764,
  0.029304541647434235,
  -0.005818665958940983,
  0.056976694613695145,
  0.007770087104290724,
  -0.0017468768637627363,
  0.001749835442751646,
  -0.0029992968775331974,
  0.0088743781670928,
  0.009725399315357208,
  -0.004532261751592159,
  0.013797883875668049,
  -0.035729311406612396,
  -0.006953231990337372,
  -0.0016205953434109688,
  0.00

In [39]:
# retrieval

def query_embedding(query_text):
    return embeddings.embed_query(query_text, task_type="RETRIEVAL_QUERY")

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def retrieve(query_text, top_k):
    results = sorted(
        vector_store,
        key=lambda x: cosine_similarity(query_embedding(query_text), x["embedding"]),
        reverse=True
        )
    
    top_chunks = results[:top_k]

    return top_chunks
    

In [41]:
query_text = """Symptoms: chest pain, shortness of breath
Lab results: ECG ST elevation, troponin high"""

retrv = retrieve(query_text, 6)

In [42]:
for items in retrv:
    print(items["text"], items["metadata"])

Vitals: BP: 170/105, HR: 110
Lab results: ECG: ST elevation, Troponin: Elevated {'patient_id': 'P_10010', 'chunk_type': 'clinical', 'department': 'Cardiology'}
Vitals: BP: 140/90, HR: 92
Lab results: ECG: Normal, Troponin: Normal {'patient_id': 'P_10001', 'chunk_type': 'clinical', 'department': 'Cardiology'}
Symptoms: Chest pain radiating to left arm, Shortness of breath, Nausea {'patient_id': 'P_10010', 'chunk_type': 'symptoms', 'department': 'Cardiology'}
Doctor Assesment: Acute myocardial infarction suspected {'patient_id': 'P_10010', 'chunk_type': 'assessment', 'department': 'Cardiology'}
Vitals: BP: 118/76, HR: 102
Lab results: ECG: Normal {'patient_id': 'P_10018', 'chunk_type': 'clinical', 'department': 'Psychiatry'}
Symptoms: Chest tightness for 2 days, Mild shortness of breath, Sweating {'patient_id': 'P_10001', 'chunk_type': 'symptoms', 'department': 'Cardiology'}


In [43]:
patient_groups = defaultdict(list)

for r in retrv:
    patient_groups[r["metadata"]["patient_id"]].append(r)

In [44]:
# getting highest similarity

highest_similarity_patient = ""
highest_similarity_patient_department = ""
c = 0
for pat_id in patient_groups.keys():
    #print(pat_id)
    #print(len(patient_groups[pat_id]))
    #print(patient_groups[pat_id][0]["metadata"]["department"])
    if len(patient_groups[pat_id]) > c:
        highest_similarity_patient, highest_similarity_patient_department = pat_id, patient_groups[pat_id][0]["metadata"]["department"]
        c = len(patient_groups[pat_id])

print("highest_similarity_patient: ", highest_similarity_patient, ", highest_similarity_patient_department: ", highest_similarity_patient_department)


highest_similarity_patient:  P_10010 , highest_similarity_patient_department:  Cardiology


In [50]:
# getting hospital list, doctors and available slots

hospital_list = json.loads(open(os.path.join("Hospital_Details", "hospital_list.json")).read())

selected_hospitals = []
for hospital in hospital_list:
    if highest_similarity_patient_department in hospital["department"]:
        selected_hospitals.append(hospital["hospital"])

selected_hospitals_doctors = []
for hospital in selected_hospitals:
    specific_hospital = {}
    specific_hospital["hospital_name"] = hospital
    file_content = json.loads(open(os.path.join("Hospital_Details", "hospital_" + hospital.lower() + ".json")).read())
    for department in file_content:
        if department["Department"] == highest_similarity_patient_department:
            specific_hospital["doctor_time_slots"] = department["Doctor_List"]
            break
    selected_hospitals_doctors.append(specific_hospital)

selected_hospitals_doctors


[{'hospital_name': 'CCC',
  'doctor_time_slots': [{'Doctor_Name': 'C',
    'Available_Time_Slots': ['02-01-2026 14:00', '02-01-2026 15:00']}]},
 {'hospital_name': 'DDD',
  'doctor_time_slots': [{'Doctor_Name': 'D',
    'Available_Time_Slots': ['02-01-2026 14:00', '02-01-2026 15:00']}]}]